In [ ]:
from numpy import inf
from docxtpl import DocxTemplate

template = DocxTemplate('portfolio.docx')

In [ ]:
class portfolio():
    def __init__(self, balance = 100000.0, acount_type = 'standard', account_currency = 'usd', leverage = 0.01):
        ## constants
        lot_sizes = {"standard":100000.0, "mini":10000.0, "micro":1000.0, "nono":100.0}
        
        self._balance = balance
        self._equity = balance
        self._margin = 0.0
        self._free_margin = balance
        self._margin_level = inf
        self._lot_size = lot_sizes[acount_type.strip().lower()]
        self._account_currency = account_currency.strip().lower()
        self._leverage = leverage
        self._closed_profit = 0.0
        self._open_profit = 0.0
        self._open_loss = 0.0
        self._risk_reward_ratio = 0.0
        self._trades = {}
        
    def get_equity(self, balance, profit_loss):
        return balance - profit ## balance - closed profit
    
    def get_lot_value(self, price, lot_size):
        return price * lot_size
    
    def get_margin(self, lot_value, leverage, volume):
        return lot_value * leverage * volume
    
    def get_free_margin(self, equity, margin):
        return equity - margin
    
    def margin_level(self, equity, margin):
        return (equity / margin) * 100
    
    def get_profit(self, open_lot_value, close_lot_value, volume, order_type):
        order_types = {"sell":-1.0, "buy":1.0} ## must set profit to (close - open)
        return (close_lot_value - open_lot_value) * volume * order_types[order_type.strip().lower()]
    
    def _add_trade(self, ticket, base_currency, quote_currency, volume, SL, TP, order_type, current_prices):
        
        if not(base_currency != self._account_currency and quote_currency != self._account_currency):
            # FIRST GET OPEN PRICES
            price_types = {"sell":"bid", "buy":"ask"}
            open_price = current_prices[base_currency + quote_currency][price_types[order_type]]
            print(type(open_price))

            # NOW UPDATE PORTFOLIO STATE
            ## 1 ## Margin
            open_lot_value = self.get_lot_value(open_price, self._lot_size)
            if base_currency == self._account_currency:
                self._margin += self.get_margin(lot_value = open_lot_value, leverage = self._leverage, volume = volume) / open_price
            else:
                self._margin += self.get_margin(lot_value = open_lot_value, leverage = self._leverage, volume = volume)
            
            self._free_margin = self._equity - self._margin
            self._margin_level = abs((self._equity - self._margin) * 100)
            
            ## 2 ## Profit
            close_lot_value_SL = self.get_lot_value(SL, self._lot_size) ## base/quote lot value at SL close
            close_lot_value_TP = self.get_lot_value(TP, self._lot_size) ## base/quote lot value at TP close
            
            trade_profit = self.get_profit(open_lot_value, close_lot_value_TP, volume, order_type)
            trade_loss = self.get_profit(open_lot_value, close_lot_value_SL, volume, order_type)
            
            self._open_profit += trade_profit
            self._open_loss += trade_loss
            
            ## 3 ## Risk/Reward ratio
            risk_reward_ratio = abs(trade_loss / trade_profit)
            self._risk_reward_ratio = abs(self._open_loss / self._open_profit)

            self._trades[ticket] = {
                'ticket': ticket,
                'order_type': order_type,
                'volume': volume,
                'base_currency': base_currency,
                'quote_currency': quote_currency,
                'open_price': open_price,
                'SL': SL,
                'TP': TP,
                'trade_profit': trade_profit,
                'trade_loss': trade_loss,
                'risk_reward_ratio': risk_reward_ratio
            }
        
            
        else:
            print("Trade expected profit cannot be calculated! Currency pair must contain account currency.")
        

In [ ]:
print('Enter portfolio information')

blnc = float(input('Balance: '))
acc_type = input('Account type [standard, mini, micro, nano] : ')
acc_crncy = input('Account currency [usd, eur, cad, aud, ...] : ')
lvrg = float(input('Account leverage [as fraction] : '))



In [ ]:
pf = portfolio(balance = blnc, acount_type = acc_type, account_currency = acc_crncy, leverage = lvrg)
print('Porfolio created succefully!')

In [ ]:

ticket = int(input('Ticket: '))
base_currency = input('base_currency: ')
quote_currency = input('quote_currency: ')
volume = float(input('volume: '))
SL = float(input('SL: '))
TP = float(input('TP: '))
order_type = input('order_type: ')
bid = float(input('bid: '))
ask = float(input('ask: '))
symbol = base_currency+quote_currency
current_prices = {symbol:{'bid':bid, 'ask':ask}}

In [ ]:
pf._add_trade(ticket, base_currency, quote_currency, volume, SL, TP, order_type, current_prices)

In [ ]:
#Render automated report
template = DocxTemplate('portfolio.docx')
template_params = pf.__dict__
template_params.update(pf.__dict__['_trades'][ticket])
template.render(template_params)
template.save('New_portfolio.docx')

In [ ]:
print(template_params)